# 5. Deploying Machine Learning models 

We'll use the same model we trained and evaluated previously - the churn prediction model. Now we'll deploy it as a web service.

## 5.1 Intro / Session overview
We need to put the model that lives in our Jupyter Notebook into production, so other services can use the model to make decisions based on the output of our model.

Suppose we have a service for running marketing campaigns. For each customer, it needs to determine the probability of churn, and if it's high enough, it will send a promotional email with discounts. This service needs to use our model to decide whether it should send an email. 

What we will cover this week: 
* Saving models with Pickle
* Serving models with Flask
* Managing dependencies with Pipenv
* Making the service self-contained with Docker
* Deploying it to the cloud using AWS Elastic Beanstalk

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [3]:
# df = pd.read_csv('data-week-3.csv')
df = pd.read_csv(data)

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [4]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [5]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [10]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=3000)
    model.fit(X_train, y_train)
    
    return dv, model

In [11]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [12]:
C = 1.0
n_splits = 5

In [13]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.842 +- 0.007


In [14]:
scores

[np.float64(0.8443806862337213),
 np.float64(0.8449563799496754),
 np.float64(0.83351796106763),
 np.float64(0.8347649005563726),
 np.float64(0.8517892441404411)]

In [15]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

np.float64(0.8583598751990639)

## 5.2 Saving and loading the model

* Saving the model to pickle
* Loading the model from pickle
* Turning our notebook into a Python script

To be able to use it outside of our notebook, we need to save it, and then later, another process can load and use it.

Pickle is a serialization/deserialization module that's already built into Python: using it, we can save an arbitrary Python object (with a few exceptions) to a file. Once we have a file, we can load the model from there in a different process.

Install the library with the command `pip install pickle-mixin` if you don't have it.

To save the model, we first import the `pickle` module, and then use the `dump` function:
  ```python
  import pickle

  with open('model.bin', 'wb') as f_out:  # 'wb' means write binary
      pickle.dump((dict_vectorizer, model), f_out)
  ```
  
To use the model, we need to open the binary file we saved and load the model using the `load` function.

  ```python
  import pickle

  with open('model.bin', 'rb') as f_in:  # 'rb' means read binary
      # Note: Never open a binary file from an untrusted source!
      dict_vectorizer, model = pickle.load(f_in)
  ```

#### Save the model

In [16]:
import pickle

In [17]:
output_file = f'model_C={C}.bin'

In [18]:
output_file

'model_C=1.0.bin'

In [19]:
# Write to the file in binary
f_out = open(output_file, 'wb') 
pickle.dump((dv, model), f_out)
f_out.close()

In [20]:
# !ls -lh *.bin

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
with open(output_file, 'wb') as f_out: 
    pickle.dump((dv, model), f_out)

#### Load the model

In [24]:
import pickle

In [25]:
input_file = 'model_C=1.0.bin'

Be careful when specifying the mode. Accidentally specifying an incorrect mode may result in data loss: if you open an existing file with the `w` mode instead of `r`, it will overwrite the content.

Also, unpickling objects found on the internet is not secure: it can execute arbitrary code on you machine. Use it only for things you trust and things you saved yourself.

In [4]:
with open(input_file, 'rb') as f_in: 
    dv, model = pickle.load(f_in)

In [26]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=3000))

Notice that we did not import scikit-learn but we need to have scikit-learn installed on our computer for this to work. Otherwise, it will complain not knowing what this is (referring to these classes) when we try to load the pickle file and this is because scikit-learn is not installed on our computer. 

In [27]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [28]:
# Turn this customer into feature matrix
X = dv.transform([customer])
X

array([[ 1.  ,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,
         0.  ,  1.  ,  0.  ,  0.  , 29.85,  0.  ,  1.  ,  0.  ,  0.  ,
         0.  ,  1.  ,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,
         0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ,
         0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  , 29.85]])

In [36]:
model.predict_proba(X)

array([[0.37255464, 0.62744536]])

In [37]:
model.predict_proba(X)[:, 1]

array([0.62744536])

In [38]:
# Get the probability that this particular customer is going to churn
y_pred = model.predict_proba(X)[0, 1]
y_pred

np.float64(0.6274453618230489)

In [39]:
print('input:', customer)
print('output:', y_pred)

input: {'gender': 'female', 'seniorcitizen': 0, 'partner': 'yes', 'dependents': 'no', 'phoneservice': 'no', 'multiplelines': 'no_phone_service', 'internetservice': 'dsl', 'onlinesecurity': 'no', 'onlinebackup': 'yes', 'deviceprotection': 'no', 'techsupport': 'no', 'streamingtv': 'no', 'streamingmovies': 'no', 'contract': 'month-to-month', 'paperlessbilling': 'yes', 'paymentmethod': 'electronic_check', 'tenure': 1, 'monthlycharges': 29.85, 'totalcharges': 29.85}
output: 0.6274453618230489


This way, we can load the model and apply it to the customer we specified in the script. 

Of course, we aren't going to manually put the information about customers in the script. In the next section, we'll cover a more practical approach where we will be putting the model into a web service. 

## 5.3 Web services: introduction to Flask

* Writing a simple ping/pong app
* Querying it with `curl` and browser

Making requests

In [40]:
import requests

In [41]:
url = 'http://localhost:9696/predict'

In [42]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'two_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [ ]:
response = requests.post(url, json=customer).json()

In [39]:
response

{'churn': True, 'churn_probability': 0.5133820686195286}

In [27]:
if response['churn']:
    print('sending email to', 'asdx-123d')

sending email to asdx-123d
